In [1]:
import pandas as pd
import numpy as np
import statistics
import matplotlib.pyplot as plt
import seaborn as sns
import pyreadstat

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, MinMaxScaler, StandardScaler, RobustScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import fdrcorrection
import scipy.stats as stats

In [2]:
# base loc
W_LOC = "/panfs/jay/groups/21/thyagara/sesha059/LLFS/" 

In [5]:
# getting ENSG ids
ENSG_list = pd.read_csv(rf'{W_LOC}/share_llfs_data/data/fltr_feature_id_gene_20220608.csv')

# read text file into pandas DataFrame
gene_exp_df = pd.read_csv(rf"{W_LOC}/share_llfs_data/data/norm_fltr_counts_gene_20220608.csv")
gene_exp_df.replace({0: 0.01}, inplace=True)
gene_exp_df = np.log2(gene_exp_df)

# taking transpose to have genes as columns
gene_exp_df.index = list(ENSG_list['gene_id'].str.split('.').str.get(0))
gene_exp_df.reset_index(inplace=True)
gene_exp_df = gene_exp_df.rename(columns = {'index': 'gene_id'})
gene_exp_df = gene_exp_df.drop_duplicates(subset=['gene_id'])

# transposing to have genes as columns
gene_exp_df = gene_exp_df.set_index('gene_id').T.reset_index(names='subject_id')
gene_exp_df['subject'] = gene_exp_df['subject_id'].str.extract(r'([0-9]+)').astype('int64')
gene_exp_df.index.name = "index"
gene_exp_df = gene_exp_df[gene_exp_df['subject_id'].str.contains('v1')]
gene_exp_df

gene_id,subject_id,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036,ENSG00000001084,ENSG00000001167,ENSG00000001460,...,ENSG00000288701,ENSG00000288703,ENSG00000288709,ENSG00000288710,ENSG00000288719,ENSG00000288720,ENSG00000288721,ENSG00000288722,ENSG00000288725,subject
index,,,,,,,,,,,,,,,,,,,,,
0,s12207_v1,7.689375,7.693279,6.998688,11.978293,5.208340,6.560919,7.835121,9.218504,5.582254,...,11.180271,1.990677,-6.643856,4.267703,5.575396,5.176915,2.796434,9.391705,-6.643856,12207
2,s12507_v1,7.643694,8.217030,5.863754,11.848494,4.607804,6.601791,7.159700,9.346141,6.539232,...,11.505632,3.513690,-6.643856,4.731029,6.024259,4.257513,3.641354,9.157698,-6.643856,12507
4,s14422_v1,7.252573,7.435521,6.472427,12.763856,4.321318,6.673795,7.738564,9.002638,5.436328,...,11.598446,2.812865,8.648266,2.998396,5.962228,4.386505,2.609369,9.061389,-6.643856,14422
6,s12781_v1,7.230563,7.515301,5.798814,12.371540,3.889964,6.701391,7.581444,9.094431,5.995712,...,11.355487,1.034640,-6.643856,4.571009,5.649209,5.220235,4.427266,9.159800,-6.643856,12781
8,s12219_v1,7.538512,7.702181,6.633136,11.969808,6.468648,6.677696,7.797105,8.854752,5.262821,...,11.315198,2.940995,-6.643856,2.731601,5.777339,5.025051,3.583264,9.579825,2.134164,12219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,s20541_v1,7.764531,8.003844,6.463782,12.151024,5.488537,6.955611,7.836227,8.899080,5.835839,...,10.935111,2.311039,-6.643856,3.732045,5.507897,5.218133,3.846081,9.110405,-6.643856,20541
2267,s20217_v1,7.823998,8.143098,6.865931,11.788210,5.558674,6.601328,7.541696,9.242922,6.217791,...,11.082158,3.459872,7.097763,3.883822,5.801110,5.581985,2.417692,9.080759,-0.446369,20217
2270,s7623_v1,8.445680,8.188583,6.818076,11.874726,5.200729,7.052043,8.082204,9.334250,5.770583,...,10.508532,3.698077,-6.643856,5.472801,6.020803,5.350283,3.562870,9.241199,2.698937,7623


In [4]:
### Combining other clinical datasets

# triplet file for family related info
triplet_v3, meta = pyreadstat.read_sas7bdat(rf'{W_LOC}/LLFS datasets/triplet_visit3.sas7bdat')
triplet_v3['subset'] = triplet_v3['gen']
triplet_v3.loc[(triplet_v3['gen'] == 2) & (triplet_v3['relative'] == 0) & (triplet_v3['control'] == 0), 'subset'] = 'pro-spouses'
triplet_v3.loc[(triplet_v3['gen'] == 2) & (triplet_v3['relative'] == 1), 'subset'] = 'proband'
triplet_v3.loc[(triplet_v3['gen'] == 3) & (triplet_v3['control'] == 0), 'subset'] = 'offspring'
triplet_v3.loc[(triplet_v3['gen'] == 3) & (triplet_v3['control'] == 1), 'subset'] = 'off-spouses'
triplet_v3 = triplet_v3[triplet_v3['subset'].isin(['pro-spouses', 'proband', 'offspring', 'off-spouses'])]
triplet_v3 = triplet_v3[['subject', 'sex', 'Deceased', 'control','gen', 'subset']]
print(triplet_v3.shape)

# bloodall file for blood based biomarkers
bloodall, meta = pyreadstat.read_sas7bdat(rf'{W_LOC}/LLFS datasets/bloodall_20240305.sas7bdat')
bloodall['CMV_Serostatus'] = bloodall['cmvgintc'].replace({'Reactive': 'CMV Positive', 'Non-Reactive': 'CMV Negative', 'Borderline': 'CMV Negative'})
bloodall_v1 = bloodall[bloodall['CMV_Serostatus'].isin(['CMV Positive', 'CMV Negative'])] # CMV status is present only for v1 participants

immune_cells = [
                'Tcells', 'HelperT', 'CytotoxicT', 
                'Helper_Naive', 'Helper_Effector', 'Helper_Effector_Memory', 'Helper_Central_Memory', 
                'Cytotoxic_Naive', 'Cytotoxic_Effector', 'Cytotoxic_Effector_memory', 'Cytotoxic_Central_Memory', 'neut'
               ]

bloodall_v1 = bloodall_v1[['subject', 'wbc', 'alym', 'CMV_Serostatus', 'hscrp', 'il6'] + immune_cells]
bloodall_v1[immune_cells] = bloodall_v1[immune_cells]/ 100
bloodall_v1['Tcells_cnt'] = bloodall_v1['Tcells'] * bloodall_v1['alym']
print(bloodall_v1.shape)

# getting the demographic information
sdiall, meta = pyreadstat.read_sas7bdat(rf'{W_LOC}/LLFS datasets/Phase I Phenotype/sdiall.sas7bdat')
sdiall = sdiall[sdiall['visitcode'] == 1]
sdiall = sdiall[['subject', 'fc', 'race1', 'educ', '_AGE_REVISED']]
sdiall['fc'] = sdiall['fc'].replace({'PT': 'US', 'BU': 'US', 'NY': 'US'})
sdiall['educ_years'] = sdiall['educ']
sdiall['educ'] = sdiall['educ'].replace({0: 'Some Schooling', 
                                           1: 'Some Schooling', 
                                           2: 'Some Schooling',
                                           3: 'Some Schooling',
                                           4: 'Some Schooling',
                                           5: 'Some Schooling', 
                                           6: 'Some Schooling', 
                                           7: 'Some Schooling', 
                                           8: 'Some Schooling', 
                                           9: 'Some Schooling', 
                                           10: 'Some Schooling', 
                                           11: 'Some Schooling', 
                                           12: 'High School', 
                                           13: 'Some College',
                                           14: 'Some College', 
                                           15: 'Some College', 
                                           16: 'College Graduate', 
                                           17: 'Post College'})
sdiall = sdiall.rename(columns = {'_AGE_REVISED': 'age', 'race1': 'race'})
print(sdiall.shape)

# getting the health information
bphr, meta = pyreadstat.read_sas7bdat(rf'{W_LOC}LLFS datasets/Phase I Phenotype/bphr.sas7bdat')
bphr = bphr[['subject', '_BMI']]

bphr = bphr.rename(columns = {'_BMI': 'bmi'})
print(bphr.shape)

# merging the datasets
res_df = triplet_v3.merge(bloodall_v1, how="inner", on=['subject'])
res_df = res_df.merge(sdiall, how="left", on=['subject'])
res_df = res_df.merge(bphr, how="left", on=['subject'])
res_df

/tmp/ipykernel_554965/961376644.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'pro-spouses' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  triplet_v3.loc[(triplet_v3['gen'] == 2) & (triplet_v3['relative'] == 0) & (triplet_v3['control'] == 0), 'subset'] = 'pro-spouses'


(17772, 6)
(4445, 19)
(4938, 6)
(4851, 2)


,subject,sex,Deceased,control,gen,subset,wbc,alym,CMV_Serostatus,hscrp,...,Cytotoxic_Effector_memory,Cytotoxic_Central_Memory,neut,Tcells_cnt,fc,race,educ,age,educ_years,bmi
0,8.0,2.0,1.0,0.0,3.0,offspring,4.8,1.4,CMV Positive,1.219999,...,0.133835,0.010572,0.579,1.138818,DK,1.0,Some Schooling,69.0,3.0,26.5
1,10.0,2.0,1.0,0.0,2.0,proband,5.6,1.9,CMV Positive,3.830000,...,0.028085,0.000092,0.530,1.165931,DK,1.0,Some Schooling,103.0,3.0,NaN
2,27.0,2.0,0.0,0.0,2.0,proband,8.1,3.7,CMV Positive,3.500000,...,0.448194,0.089418,0.451,2.925801,DK,1.0,Some Schooling,86.0,4.0,27.4
3,46.0,2.0,1.0,0.0,2.0,proband,5.0,2.0,CMV Positive,1.030000,...,0.099100,0.015800,0.490,1.808000,DK,1.0,Some Schooling,92.0,3.0,19.0
4,47.0,2.0,0.0,1.0,3.0,off-spouses,7.3,2.3,CMV Negative,0.850000,...,0.149000,0.052900,0.560,1.890600,DK,1.0,Some Schooling,58.0,5.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4440,32599.0,1.0,0.0,0.0,3.0,offspring,5.6,1.7,CMV Negative,1.440000,...,NaN,NaN,0.600,NaN,US,NaN,NaN,61.0,NaN,NaN
4441,32640.0,2.0,0.0,0.0,2.0,proband,6.9,1.9,CMV Negative,0.240000,...,0.325104,0.027150,0.630,1.461504,US,1.0,Some College,57.0,15.0,26.0
4442,32643.0,2.0,0.0,0.0,2.0,proband,5.2,1.5,CMV Positive,0.430000,...,NaN,NaN,0.630,NaN,US,1.0,Some College,57.0,15.0,26.8
4443,32646.0,2.0,0.0,0.0,2.0,proband,6.2,2.7,CMV Negative,5.480000,...,0.246198,0.023159,0.480,1.959203,US,1.0,Some College,49.0,15.0,33.7


In [6]:
LLFS_df = res_df.merge(gene_exp_df, how="inner", on=['subject'])

LLFS_df.shape

(1302, 18163)

In [7]:
LLFS_df.to_csv('LLFS_planB_data.csv', index=False)

### Finding genes common between HRS and LLFS

In [9]:
# common coexpression modules from group lasso
common_modules = pd.read_csv('/panfs/jay/groups/21/thyagara/sesha059/Plan B/New/common_modules.csv')
common_modules['gene_id'] = common_modules['gene_id'].str.split('.').str.get(0)
common_modules['colors'].value_counts()

# finding overlapping genes
common_modules2 = common_modules.loc[common_modules['gene_id'].isin(LLFS_df.columns), :]
common_modules2['colors'].value_counts()

colors
brown    460
black    186
pink     176
Name: count, dtype: int64

In [10]:
common_modules2.to_csv('LLFS_HRS_overlap_module_genes.csv', index = False)

In [11]:
LLFS_df.columns[:50]

Index(['subject', 'sex', 'Deceased', 'control', 'gen', 'subset', 'wbc', 'alym',
       'CMV_Serostatus', 'hscrp', 'il6', 'Tcells', 'HelperT', 'CytotoxicT',
       'Helper_Naive', 'Helper_Effector', 'Helper_Effector_Memory',
       'Helper_Central_Memory', 'Cytotoxic_Naive', 'Cytotoxic_Effector',
       'Cytotoxic_Effector_memory', 'Cytotoxic_Central_Memory', 'neut',
       'Tcells_cnt', 'fc', 'race', 'educ', 'age', 'educ_years', 'bmi',
       'subject_id', 'ENSG00000000419', 'ENSG00000000457', 'ENSG00000000460',
       'ENSG00000000938', 'ENSG00000000971', 'ENSG00000001036',
       'ENSG00000001084', 'ENSG00000001167', 'ENSG00000001460',
       'ENSG00000001461', 'ENSG00000001497', 'ENSG00000001561',
       'ENSG00000001629', 'ENSG00000001630', 'ENSG00000001631',
       'ENSG00000002016', 'ENSG00000002330', 'ENSG00000002549',
       'ENSG00000002586'],
      dtype='object')

In [12]:
LLFS_df['CMV_Serostatus'].value_counts()

CMV_Serostatus
CMV Positive    749
CMV Negative    553
Name: count, dtype: int64